# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26

def idx_to_onehot(x, k=N+1):
  ones = torch.sparse.torch.eye(k).to(x.device)
  """ Converts the generated integers to one-hot vectors """
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])

class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=3, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [2]:
import torch
from torch import nn

class GRUMemory(torch.nn.Module):

    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size=N+1, hidden_size=hidden_size, batch_first=True)
        self.decoder = nn.Linear(hidden_size, N+1)

    def forward(self, x):
        output, _ = self.gru(x)
        logits = self.decoder(output)
        return logits

    @torch.no_grad()
    def test_run(self, s):
        indices = torch.tensor([ord(c) - ord('a') + 1 for c in s], dtype=torch.long)
        indices = indices.to(next(self.parameters()).device)
        one_hot = idx_to_onehot(indices).unsqueeze(0)

        logits = self.forward(one_hot)
        preds = logits.argmax(dim=-1).squeeze(0)
        idx_to_char = lambda idx: ' ' if idx == 0 else chr(idx + ord('a') - 1)
        result = ''.join([idx_to_char(idx.item()) for idx in preds])
        return result


## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [3]:
def test_model(model):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [4]:
import time
from tqdm.notebook import tqdm
from torch import nn
start_time = time.time()


def train_model(model, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    n_epochs = 5
    model.train()
    for epoch in range(n_epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(tqdm(dataloader)):
            data = data.to(device)
            target = target.to(device)

            data_one_hot = idx_to_onehot(data)


            logits = model(data_one_hot)

            logits_flat = logits.view(-1, N+1)
            target_flat = target.view(-1)

            loss = criterion(logits_flat, target_flat)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {avg_loss:.4f}')


import time


HIDDEN_SIZE = 128
BATCH_SIZE = 512

start_time = time.time()

model = GRUMemory(HIDDEN_SIZE)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)

train_model(model, dataloader)

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
print(accuracy)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.0911


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [2/5], Loss: 0.2439


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [3/5], Loss: 0.0139


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [4/5], Loss: 0.0048


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [5/5], Loss: 0.0025
1.0
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [5]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [6]:
from torch import nn

class VariableDelayGRUMemory(nn.Module):
    def __init__(self, hidden_size, max_delay, delay_embedding_dim=64, num_layers=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.max_delay = max_delay
        self.delay_embedding_dim = delay_embedding_dim
        self.num_layers = num_layers

        self.delay_embedding = nn.Embedding(max_delay + 1, delay_embedding_dim)


        self.delay_to_h0 = nn.Linear(delay_embedding_dim, hidden_size * num_layers)


        self.gru = nn.GRU(
                  input_size=N + 1 + delay_embedding_dim,
                  hidden_size=hidden_size,
                  num_layers=num_layers,
                  batch_first=True,
              )

        self.decoder = nn.Linear(hidden_size, N + 1)


    def forward(self, x, delays):

      delay_embed = self.delay_embedding(delays)

      h0 = self.delay_to_h0(delay_embed)
      h0 = h0.view(self.num_layers, x.size(0), self.hidden_size)

      seq_length = x.size(1)
      delay_embed_expanded = delay_embed.unsqueeze(1).expand(-1, seq_length, -1)


      x_combined = torch.cat((x, delay_embed_expanded), dim=-1)


      output, _ = self.gru(x_combined, h0)


      logits = self.decoder(output)

      return logits



    @torch.no_grad()
    def test_run(self, s, delay):

        indices = torch.tensor([ord(c) - ord('a') + 1 for c in s], dtype=torch.long)
        indices = indices.to(next(self.parameters()).device)


        one_hot = idx_to_onehot(indices).unsqueeze(0)

        delays = torch.tensor([delay], dtype=torch.long).to(one_hot.device)

        logits = self.forward(one_hot, delays)

        preds = logits.argmax(dim=-1).squeeze(0)

        idx_to_char = lambda idx: ' ' if idx == 0 else chr(idx + ord('a') - 1)
        result = ''.join([idx_to_char(idx.item()) for idx in preds])

        return result


In [7]:
def test_variable_delay_model(model, max_delay=12):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  delay = random.randint(0, max_delay)

  total = 0
  correct = 0
  for i in range(1000):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s, delay)
    for c1, c2 in zip(s if delay == 0 else s[:-delay], result[delay:]):
      correct += int(c1 == c2)
    total += len(s) - delay

  return correct / total

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [8]:
import time
start_time = time.time()

MAX_DELAY = 12
SEQ_LENGTH = 25


from tqdm.notebook import tqdm
from torch import nn

HIDDEN_SIZE=None # TODO
BATCH_SIZE=None #TODO


# Set hyperparameters
MAX_DELAY = 12
SEQ_LENGTH = 25
HIDDEN_SIZE = 512
DELAY_EMBEDDING_DIM = 64
BATCH_SIZE = 256
NUM_LAYERS = 2
LEARNING_RATE = 5*1e-4
N_EPOCHS = 5

model = VariableDelayGRUMemory(
    hidden_size=HIDDEN_SIZE,
    max_delay=MAX_DELAY,
    delay_embedding_dim=DELAY_EMBEDDING_DIM,
    num_layers=NUM_LAYERS,
)
ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, size=600000, seq_length=SEQ_LENGTH)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)


def train_variable_delay_model(model, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(N_EPOCHS):
        total_loss = 0
        for batch_idx, (seq, delays, result) in enumerate(tqdm(dataloader)):

            seq = seq.to(device)
            delays = delays.to(device)
            result = result.to(device)


            x = idx_to_onehot(seq)


            logits = model(x, delays)


            logits_flat = logits.view(-1, N + 1)
            result_flat = result.view(-1)
            loss = criterion(logits_flat, result_flat)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)


model = VariableDelayGRUMemory(hidden_size=HIDDEN_SIZE, max_delay=MAX_DELAY)
ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, size=600000, seq_length=SEQ_LENGTH)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)
train_variable_delay_model(model, dataloader)

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
accuracy = test_variable_delay_model(model, max_delay=MAX_DELAY)
print(accuracy)
assert accuracy > 0.95, 'accuracy is too low'
print('tests passed')

  0%|          | 0/2344 [00:00<?, ?it/s]

  0%|          | 0/2344 [00:00<?, ?it/s]

  0%|          | 0/2344 [00:00<?, ?it/s]

  0%|          | 0/2344 [00:00<?, ?it/s]

  0%|          | 0/2344 [00:00<?, ?it/s]

0.9955323901712584
tests passed
